Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
data_dir = '/home/david/Research/udacity/tensorflow'
pickle_file = os.path.join(data_dir, 'notMNIST_filtered.pickle')

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (358551, 28, 28) (358551,)
Validation set (19620, 28, 28) (19620,)
Test set (14638, 28, 28) (14638,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (358551, 784) (358551, 10)
Validation set (19620, 784) (19620, 10)
Test set (14638, 784) (14638, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
batch_size = 128
beta = 0.005

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) +
    beta * tf.nn.l2_loss(weights)
  )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf_test_dataset, weights) + biases)

In [23]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 33.966309
Minibatch accuracy: 8.6%
Validation accuracy: 9.3%
Test accuracy: 9.7%
Minibatch loss at step 500: 1.643384
Minibatch accuracy: 85.2%
Validation accuracy: 66.7%
Test accuracy: 72.8%
Minibatch loss at step 1000: 0.644287
Minibatch accuracy: 87.5%
Validation accuracy: 70.7%
Test accuracy: 77.7%
Minibatch loss at step 1500: 0.551861
Minibatch accuracy: 83.6%
Validation accuracy: 75.5%
Test accuracy: 82.0%
Minibatch loss at step 2000: 0.975060
Minibatch accuracy: 75.8%
Validation accuracy: 72.9%
Test accuracy: 78.7%
Minibatch loss at step 2500: 0.579424
Minibatch accuracy: 85.2%
Validation accuracy: 53.6%
Test accuracy: 57.1%
Minibatch loss at step 3000: 0.524924
Minibatch accuracy: 87.5%
Validation accuracy: 58.6%
Test accuracy: 62.6%
Minibatch loss at step 3500: 0.183058
Minibatch accuracy: 96.1%
Validation accuracy: 72.6%
Test accuracy: 78.9%
Minibatch loss at step 4000: 0.378898
Minibatch accuracy: 91.4%
Validation accuracy: 77.7%
Test ac

And now for the NN:

In [7]:
def create_logits(dataset, weights1, bias1, weights2, bias2):
    """
    Creates a 2-layer network from the specified dataset,
    weights and bias. It is a shame that this is everything
    that can be re-used...
    """
    hidden = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    return tf.matmul(hidden, weights2) + biases2

batch_size = 128
H = 1024
graph = tf.Graph()
with graph.as_default():
    # Input data. Just as in the previous code cell...
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(
        tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables. Now, this is different...
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, H]))
    biases1 = tf.Variable(tf.zeros([H]))
    weights2 = tf.Variable(tf.truncated_normal([H, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # The loss
    logits = create_logits(tf_train_dataset, weights1, biases1,
                           weights2, biases2)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) +
        beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) +
                tf.nn.l2_loss(biases1) + tf.nn.l2_loss(biases2))
    )
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(create_logits(
        tf_valid_dataset, weights1, biases1, weights2, biases2))
    test_prediction = tf.nn.softmax(create_logits(
        tf_test_dataset, weights1, biases1, weights2, biases2))

In [30]:
beta = 0.001
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1904.472046
Minibatch accuracy: 8.6%
Validation accuracy: 17.7%
Test accuracy: 18.3%
Minibatch loss at step 500: 132.862778
Minibatch accuracy: 80.5%
Validation accuracy: 58.9%
Test accuracy: 63.2%
Minibatch loss at step 1000: 11.395744
Minibatch accuracy: 89.1%
Validation accuracy: 74.7%
Test accuracy: 80.7%
Minibatch loss at step 1500: 1.418802
Minibatch accuracy: 87.5%
Validation accuracy: 79.4%
Test accuracy: 85.3%
Minibatch loss at step 2000: 0.949484
Minibatch accuracy: 79.7%
Validation accuracy: 77.7%
Test accuracy: 83.2%
Minibatch loss at step 2500: 0.538684
Minibatch accuracy: 88.3%
Validation accuracy: 65.3%
Test accuracy: 70.1%
Minibatch loss at step 3000: 0.733026
Minibatch accuracy: 87.5%
Validation accuracy: 57.4%
Test accuracy: 61.8%
Minibatch loss at step 3500: 6.628024
Minibatch accuracy: 57.8%
Validation accuracy: 56.1%
Test accuracy: 59.7%
Minibatch loss at step 4000: 0.551260
Minibatch accuracy: 93.8%
Validation accuracy: 78.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
beta = 0.001
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size) % 5  # pointo desu
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1894.421387
Minibatch accuracy: 8.6%
Validation accuracy: 16.5%
Test accuracy: 17.4%
Minibatch loss at step 500: 129.939667
Minibatch accuracy: 99.2%
Validation accuracy: 22.8%
Test accuracy: 23.9%
Minibatch loss at step 1000: 10.771023
Minibatch accuracy: 99.2%
Validation accuracy: 29.8%
Test accuracy: 31.4%
Minibatch loss at step 1500: 1.056772
Minibatch accuracy: 99.2%
Validation accuracy: 38.2%
Test accuracy: 41.3%
Minibatch loss at step 2000: 0.258529
Minibatch accuracy: 99.2%
Validation accuracy: 39.8%
Test accuracy: 43.0%
Minibatch loss at step 2500: 0.188599
Minibatch accuracy: 99.2%
Validation accuracy: 40.0%
Test accuracy: 43.2%
Minibatch loss at step 3000: 0.174318
Minibatch accuracy: 100.0%
Validation accuracy: 40.1%
Test accuracy: 43.3%
Minibatch loss at step 3500: 0.172363
Minibatch accuracy: 100.0%
Validation accuracy: 40.0%
Test accuracy: 43.2%
Minibatch loss at step 4000: 0.171799
Minibatch accuracy: 99.2%
Validation accuracy: 39.9

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
def create_logits_with_dropout(dataset, weights1, bias1, weights2, bias2, keep_prob):
    """
    Creates a 2-layer network from the specified dataset,
    weights and bias. It is a shame that this is everything
    that can be re-used...
    """
    hidden = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    hidden_with_dropout = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(hidden_with_dropout, weights2) + biases2

batch_size = 128
H = 1024
graph = tf.Graph()
with graph.as_default():
    # Input data. Just as in the previous code cell...
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(
        tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32)

    # Variables. Now, this is different...
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, H]))
    biases1 = tf.Variable(tf.zeros([H]))
    weights2 = tf.Variable(tf.truncated_normal([H, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # The loss
    logits = create_logits_with_dropout(tf_train_dataset, weights1, biases1,
                                        weights2, biases2, keep_prob)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) +
        beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) +
                tf.nn.l2_loss(biases1) + tf.nn.l2_loss(biases2))
    )
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(create_logits(
        tf_valid_dataset, weights1, biases1, weights2, biases2))
    test_prediction = tf.nn.softmax(create_logits(
        tf_test_dataset, weights1, biases1, weights2, biases2))

In [17]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                 beta: 0.001, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 849.173706
Minibatch accuracy: 7.0%
Validation accuracy: 21.5%
Test accuracy: 23.0%
Minibatch loss at step 500: 208.143951
Minibatch accuracy: 67.2%
Validation accuracy: 58.9%
Test accuracy: 64.2%
Minibatch loss at step 1000: 124.243126
Minibatch accuracy: 74.2%
Validation accuracy: 64.4%
Test accuracy: 70.5%
Minibatch loss at step 1500: 74.276855
Minibatch accuracy: 78.9%
Validation accuracy: 68.9%
Test accuracy: 75.3%
Minibatch loss at step 2000: 47.997673
Minibatch accuracy: 64.8%
Validation accuracy: 63.9%
Test accuracy: 68.8%
Minibatch loss at step 2500: 31.139328
Minibatch accuracy: 79.7%
Validation accuracy: 60.1%
Test accuracy: 65.7%
Minibatch loss at step 3000: 17.102200
Minibatch accuracy: 82.0%
Validation accuracy: 62.8%
Test accuracy: 68.2%
Minibatch loss at step 3500: 10.088228
Minibatch accuracy: 95.3%
Validation accuracy: 79.3%
Test accuracy: 86.2%
Minibatch loss at step 4000: 6.321928
Minibatch accuracy: 93.0%
Validation accuracy: 8

In [20]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size) % 5
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                 beta: 0.001, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 812.357544
Minibatch accuracy: 9.4%
Validation accuracy: 16.0%
Test accuracy: 17.1%
Minibatch loss at step 500: 193.136566
Minibatch accuracy: 96.9%
Validation accuracy: 33.2%
Test accuracy: 36.2%
Minibatch loss at step 1000: 117.154518
Minibatch accuracy: 98.4%
Validation accuracy: 34.3%
Test accuracy: 37.2%
Minibatch loss at step 1500: 71.050385
Minibatch accuracy: 99.2%
Validation accuracy: 35.1%
Test accuracy: 38.0%
Minibatch loss at step 2000: 43.108307
Minibatch accuracy: 99.2%
Validation accuracy: 35.4%
Test accuracy: 38.5%
Minibatch loss at step 2500: 26.172335
Minibatch accuracy: 98.4%
Validation accuracy: 36.2%
Test accuracy: 39.4%
Minibatch loss at step 3000: 15.869926
Minibatch accuracy: 100.0%
Validation accuracy: 35.9%
Test accuracy: 38.9%
Minibatch loss at step 3500: 9.640301
Minibatch accuracy: 100.0%
Validation accuracy: 35.8%
Test accuracy: 38.6%
Minibatch loss at step 4000: 5.865158
Minibatch accuracy: 100.0%
Validation accuracy:

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [21]:
def create_logits_with_dropout(dataset, weights1, bias1, weights2, bias2, keep_prob):
    """
    Creates a 2-layer network from the specified dataset,
    weights and bias. It is a shame that this is everything
    that can be re-used...
    """
    hidden = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    hidden_with_dropout = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(hidden_with_dropout, weights2) + biases2

batch_size = 128
H = 1024
graph = tf.Graph()
with graph.as_default():
    # Input data. Just as in the previous code cell...
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(
        tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32)
    
    # Learning rate decay
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.5, global_step, 50, 0.99)

    # Variables. Now, this is different...
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, H]))
    biases1 = tf.Variable(tf.zeros([H]))
    weights2 = tf.Variable(tf.truncated_normal([H, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # The loss
    logits = create_logits_with_dropout(tf_train_dataset, weights1, biases1,
                                        weights2, biases2, keep_prob)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) +
        beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) +
                tf.nn.l2_loss(biases1) + tf.nn.l2_loss(biases2))
    )
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(create_logits(
        tf_valid_dataset, weights1, biases1, weights2, biases2))
    test_prediction = tf.nn.softmax(create_logits(
        tf_test_dataset, weights1, biases1, weights2, biases2))

In [22]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                 beta: 0.001, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 813.350586
Minibatch accuracy: 9.4%
Validation accuracy: 12.4%
Test accuracy: 12.5%
Minibatch loss at step 500: 205.384750
Minibatch accuracy: 71.1%
Validation accuracy: 53.6%
Test accuracy: 58.0%
Minibatch loss at step 1000: 133.680939
Minibatch accuracy: 73.4%
Validation accuracy: 62.3%
Test accuracy: 67.6%
Minibatch loss at step 1500: 89.108932
Minibatch accuracy: 75.8%
Validation accuracy: 66.2%
Test accuracy: 72.2%
Minibatch loss at step 2000: 69.588654
Minibatch accuracy: 70.3%
Validation accuracy: 61.3%
Test accuracy: 66.9%
Minibatch loss at step 2500: 51.380730
Minibatch accuracy: 76.6%
Validation accuracy: 60.7%
Test accuracy: 65.8%
Minibatch loss at step 3000: 33.596107
Minibatch accuracy: 83.6%
Validation accuracy: 63.6%
Test accuracy: 68.5%
Minibatch loss at step 3500: 25.362324
Minibatch accuracy: 92.2%
Validation accuracy: 79.0%
Test accuracy: 85.4%
Minibatch loss at step 4000: 20.186413
Minibatch accuracy: 88.3%
Validation accuracy: 